In [1]:
#Omid55
import requests
import pandas as pd
import pickle as pk
import csv
import time
import numpy as np
import random

import seaborn as sns
import matplotlib.pylab as plt
% matplotlib inline

In [2]:
URL = {
    'BASE': 'https://{proxy}.api.pvp.net/api/lol/{region}/{rest}',
    'STATIC_BASE': 'https://global.api.pvp.net/api/lol/static-data/{region}/v1.2/{rest}',
    
    'MATCHLIST_URL': 'v2.2/matchlist/by-summoner/{summonerId}?seasons=SEASON{season}',
    'MATCHLIST_URL_WITH_DATE': 'v2.2/matchlist/by-summoner/{summonerId}?seasons=SEASON{season}&endTime={date}',
    'MATCH_URL': 'v2.2/match/{matchId}',
    'CHAMPION_URL': 'champion/{id}?champData=all',
    'SUMMONER_URL': 'v1.4/summoner/{summonerId}',
    'SUMMONER_WINS_LOSSES_PER_CHAMPION_URL': 'v1.3/stats/by-summoner/{summonerId}/ranked?season=SEASON{season}'
}

REGIONS = {
    'north america': 'na',
    'europe west': 'euw'
}

In [3]:
class Match(object):
    
    def __init__(self):
        self.winners = []
        self.losers = []
        self.duration = -1
        

In [81]:
class RiotAPI(object):
    
    def __init__(self, api_key, region=REGIONS['north america']):
        self.api_key = api_key
        self.region = region
        self.champions = {}
#         self.champions_allinfo = {}
#         self.champions_allinfo_saved = False
        self.summoner_wins_losses = {}
        
    def _request(self, base, rest, params={}):
        while True:
            args = {'api_key': self.api_key}
            args.update(params)
            response = requests.get(
                URL[base].format(
                    rest=rest,
                    proxy=self.region,
                    region=self.region,
                ),
                params=args
            )
            #print(response.request.url)
            #time.sleep(1.2)
            res = response.json()
            if 'status' in res and 'status_code' in res['status'] and res['status']['status_code'] == 429:
                time.sleep(1.2)
            else:
                break
        return response.json()
    
    def _base_request(self, rest, params={}):
        return self._request('BASE', rest, params)
        
    def _static_request(self, rest, params={}):
        return self._request('STATIC_BASE', rest, params)
    
    # functions   
    def _get_list_of_match_ids(self, sid, season, before_this_date=None):
        if not before_this_date:
            rest = URL['MATCHLIST_URL'].format(
                summonerId=sid,
                season=season
            )
            result = self._base_request(rest)
        else:
            rest = URL['MATCHLIST_URL_WITH_DATE'].format(
                summonerId=sid,
                season=season,
                date=before_this_date-1
            )
            result = self._base_request(rest)
        if 'matches' in result:
            for match in result['matches']:
                yield match['matchId']
                
    def _get_match(self, mid):
        rest = URL['MATCH_URL'].format(
            matchId=mid
        )
        result = self._base_request(rest)
        return result
        
    # --------------------------------------------------------------------------------------
    def get_all_history_in_season_before_given_date(self, sid, selected_champion_id, season, endTime):
        stat_names = ['champLevel', 'kills', 'doubleKills', 'tripleKills', 'quadraKills', 'pentaKills', 'unrealKills', 
                     'largestKillingSpree', 'deaths', 'assists', 'totalDamageDealt', 'totalDamageDealtToChampions', 
                      'totalDamageTaken', 'largestCriticalStrike', 'totalHeal', 'minionsKilled', 'neutralMinionsKilled',
                      'neutralMinionsKilledTeamJungle', 'neutralMinionsKilledEnemyJungle', 'goldEarned', 'goldSpent',
                      'combatPlayerScore', 'objectivePlayerScore','totalPlayerScore', 'totalScoreRank', 'magicDamageDealtToChampions',
                      'physicalDamageDealtToChampions', 'trueDamageDealtToChampions', 'visionWardsBoughtInGame',
                      'sightWardsBoughtInGame', 'magicDamageDealt', 'physicalDamageDealt', 'trueDamageDealt', 'magicDamageTaken',
                      'physicalDamageTaken', 'trueDamageTaken', 'inhibitorKills', 'towerKills', 'wardsPlaced', 'wardsKilled',
                      'largestMultiKill', 'killingSprees', 'totalUnitsHealed', 'totalTimeCrowdControlDealt'
                     ]
    
    def get_all_history_in_same_season_before_this_date(self, sid, selected_champion_id, season, before_this_date):
        same_champ_win = 0
        same_champ_lose = 0
        win = 0
        lose = 0
        gold_win=0
        deaths_win=0
        kills_win=0
        assists_win=0
        champLevel_win=0
        physicalDamageDealt_win=0
        trueDamageDealt_win=0
        towerKills_win=0
        minionsKilled_win=0
        totalHeal_win=0
        unrealKills_win=0
        pentaKills_win=0                             
        magicDamageDealt_win=0
        wardsKilled_win=0
        largestCriticalStrike_win=0
        doubleKills_win=0
        tripleKills_win=0
        magicDamageDealtToChampions_win=0
        totalTimeCrowdControlDealt_win=0
        physicalDamageTaken_win=0
        largestKillingSpree_win=0
        inhibitorKills_win=0
        physicalDamageDealtToChampions_win=0                                  
        quadraKills_win=0
        goldSpent_win=0
        wardsPlaced_win=0
        killingSprees_win=0
        totalUnitsHealed_win=0
        totalPlayerScore_win=0
        magicDamageTaken_win=0
        largestMultiKill_win=0
        combatPlayerScore_win=0
        combatPlayerScore_win=0
        gold_lose=0
        deaths_lose=0
        kills_lose=0
        assists_lose=0
        champLevel_lose=0
        physicalDamageDealt_lose=0
        trueDamageDealt_lose=0
        towerKills_lose=0
        minionsKilled_lose=0
        totalHeal_lose=0
        unrealKills_lose=0
        pentaKills_lose=0                             
        magicDamageDealt_lose=0
        wardsKilled_lose=0
        largestCriticalStrike_lose=0
        doubleKills_lose=0
        tripleKills_lose=0
        magicDamageDealtToChampions_lose=0
        totalTimeCrowdControlDealt_lose=0
        physicalDamageTaken_lose=0
        largestKillingSpree_lose=0
        inhibitorKills_lose=0
        physicalDamageDealtToChampions_lose=0                                  
        quadraKills_lose=0
        goldSpent_lose=0
        wardsPlaced_lose=0
        killingSprees_lose=0
        totalUnitsHealed_lose=0
        totalPlayerScore_lose=0
        magicDamageTaken_lose=0
        largestMultiKill_lose=0
        combatPlayerScore_lose=0
        combatPlayerScore_lose=0
        
        for matchid in list(self._get_list_of_match_ids(sid, season, before_this_date)):
            match = self._get_match(matchid)
            if 'participants' in match:
                for pi in match['participantIdentities']:
                    if pi['player']['summonerId'] == sid:
                        for pa in match['participants']:
                            if pa['participantId'] == pi['participantId']:
                                if pa['teamId'] == 100:
                                    win += 1
                                    gold_win+=pa['stats']['goldEarned']
                                    deaths_win+=pa['stats']['deaths']
                                    kills_win+=pa['stats']['kills']
                                    assists_win+=pa['stats']['assists']
                                    champLevel_win+=pa['stats']['champLevel']
                                    physicalDamageDealt_win+=pa['stats']['physicalDamageDealt']
                                    trueDamageDealt_win+=pa['stats']['trueDamageDealt']
                                    towerKills_win+=pa['stats']['towerKills']
                                    minionsKilled_win+=pa['stats']['minionsKilled']
                                    totalHeal_win+=pa['stats']['totalHeal']
                                    unrealKills_win+=pa['stats']['unrealKills']
                                    pentaKills_win+=pa['stats']['pentaKills']                                    
                                    magicDamageDealt_win+=pa['stats']['magicDamageDealt']
                                    wardsKilled_win+=pa['stats']['wardsKilled']
                                    largestCriticalStrike_win+=pa['stats']['largestCriticalStrike']
                                    doubleKills_win+=pa['stats']['doubleKills']
                                    tripleKills_win+=pa['stats']['tripleKills']
                                    magicDamageDealtToChampions_win+=pa['stats']['magicDamageDealtToChampions']
                                    totalTimeCrowdControlDealt_win+=pa['stats']['totalTimeCrowdControlDealt']
                                    physicalDamageTaken_win+=pa['stats']['physicalDamageTaken']
                                    largestKillingSpree_win+=pa['stats']['largestKillingSpree']
                                    inhibitorKills_win+=pa['stats']['inhibitorKills']
                                    physicalDamageDealtToChampions_win+=pa['stats']['physicalDamageDealtToChampions']                                    
                                    quadraKills_win+=pa['stats']['quadraKills']
                                    goldSpent_win+=pa['stats']['goldSpent']
                                    wardsPlaced_win+=pa['stats']['wardsPlaced']
                                    killingSprees_win+=pa['stats']['killingSprees']
                                    totalUnitsHealed_win+=pa['stats']['totalUnitsHealed']
                                    totalPlayerScore_win+=pa['stats']['totalPlayerScore']
                                    magicDamageTaken_win+=pa['stats']['magicDamageTaken']
                                    largestMultiKill_win+=pa['stats']['largestMultiKill']
                                    combatPlayerScore_win+=pa['stats']['neutralMinionsKilled']
                                    combatPlayerScore_win+=pa['stats']['combatPlayerScore']
                                    
                                    if pa['championId'] == selected_champion_id:
                                        same_champ_win += 1
                                else:
                                    lose += 1
                                    gold_lose+=pa['stats']['goldEarned']
                                    deaths_lose+=pa['stats']['deaths']
                                    kills_lose+=pa['stats']['kills']
                                    assists_lose+=pa['stats']['assists']
                                    champLevel_lose+=pa['stats']['champLevel']
                                    physicalDamageDealt_lose+=pa['stats']['physicalDamageDealt']
                                    trueDamageDealt_lose+=pa['stats']['trueDamageDealt']
                                    towerKills_lose+=pa['stats']['towerKills']
                                    minionsKilled_lose+=pa['stats']['minionsKilled']
                                    totalHeal_lose+=pa['stats']['totalHeal']
                                    unrealKills_lose+=pa['stats']['unrealKills']
                                    pentaKills_lose+=pa['stats']['pentaKills']                                    
                                    magicDamageDealt_lose+=pa['stats']['magicDamageDealt']
                                    wardsKilled_lose+=pa['stats']['wardsKilled']
                                    largestCriticalStrike_lose+=pa['stats']['largestCriticalStrike']
                                    doubleKills_lose+=pa['stats']['doubleKills']
                                    tripleKills_lose+=pa['stats']['tripleKills']
                                    magicDamageDealtToChampions_lose+=pa['stats']['magicDamageDealtToChampions']
                                    totalTimeCrowdControlDealt_lose+=pa['stats']['totalTimeCrowdControlDealt']
                                    physicalDamageTaken_lose+=pa['stats']['physicalDamageTaken']
                                    largestKillingSpree_lose+=pa['stats']['largestKillingSpree']
                                    inhibitorKills_lose+=pa['stats']['inhibitorKills']
                                    physicalDamageDealtToChampions_lose+=pa['stats']['physicalDamageDealtToChampions']                                    
                                    quadraKills_lose+=pa['stats']['quadraKills']
                                    goldSpent_lose+=pa['stats']['goldSpent']
                                    wardsPlaced_lose+=pa['stats']['wardsPlaced']
                                    killingSprees_lose+=pa['stats']['killingSprees']
                                    totalUnitsHealed_lose+=pa['stats']['totalUnitsHealed']
                                    totalPlayerScore_lose+=pa['stats']['totalPlayerScore']
                                    magicDamageTaken_lose+=pa['stats']['magicDamageTaken']
                                    largestMultiKill_lose+=pa['stats']['largestMultiKill']
                                    combatPlayerScore_lose+=pa['stats']['neutralMinionsKilled']
                                    combatPlayerScore_lose+=pa['stats']['combatPlayerScore']
                                    if pa['championId'] == selected_champion_id:
                                        same_champ_lose += 1
                                #break    
        if win+lose==0:
            print(sid, selected_champion_id, season, before_this_date)
            return {}
        if same_champ_win+same_champ_lose:
            r = same_champ_win / float(same_champ_win+same_champ_lose)
        else:
            r = 0
        res={
            'win_ratio': win / float(win+lose),
            'games_count': win+lose,
            'same_champ_win_ratio': r,
            'same_champ_games_count': same_champ_win+same_champ_lose,
            'unrealKills_lose': unrealKills_lose,
            'pentaKills_lose': pentaKills_lose,
            'magicDamageDealt_lose': magicDamageDealt_lose,
            'wardsKilled_lose': wardsKilled_lose,
            'largestCriticalStrike_lose': largestCriticalStrike_lose,
            'doubleKills_lose': doubleKills_lose,
            'tripleKills_lose': tripleKills_lose,
            'magicDamageDealtToChampions_lose': magicDamageDealtToChampions_lose,
            'totalTimeCrowdControlDealt_lose': totalTimeCrowdControlDealt_lose,
            'physicalDamageTaken_lose': physicalDamageTaken_lose,
            'largestKillingSpree_lose': largestKillingSpree_lose,
            'inhibitorKills_lose': inhibitorKills_lose,
            'physicalDamageDealtToChampions_lose': physicalDamageDealtToChampions_lose,
            'quadraKills_lose': quadraKills_lose,
            'goldSpent_lose': goldSpent_lose,
            'wardsPlaced_lose': wardsPlaced_lose,
            'killingSprees_lose': killingSprees_lose,
            'totalUnitsHealed_lose': totalUnitsHealed_lose,
            'totalPlayerScore_lose': totalPlayerScore_lose,
            'magicDamageTaken_lose': magicDamageTaken_lose,
            'largestMultiKill_lose': largestMultiKill_lose,
            'combatPlayerScore_lose': combatPlayerScore_lose,
            'combatPlayerScore_lose': combatPlayerScore_lose,
            'gold_lose': gold_lose,
            'deaths_lose': deaths_lose,
            'kills_lose': kills_lose,
            'assists_lose': assists_lose,
            'champLevel_lose': champLevel_lose,
            'physicalDamageDealt_lose': physicalDamageDealt_lose,
            'trueDamageDealt_lose': trueDamageDealt_lose,
            'towerKills_lose': towerKills_lose,
            'minionsKilled_lose': minionsKilled_lose,
            'totalHeal_lose': totalHeal_lose,
            'gold_win': gold_win,
            'deaths_win': deaths_win,
            'kills_win': kills_win,
            'assists_win': assists_win,
            'champLevel_win': champLevel_win,
            'physicalDamageDealt_win': physicalDamageDealt_win,
            'trueDamageDealt_win': trueDamageDealt_win,
            'towerKills_win': towerKills_win,
            'minionsKilled_win': minionsKilled_win,
            'totalHeal_win': totalHeal_win,
            'unrealKills_win': unrealKills_win,
            'pentaKills_win': pentaKills_win,
            'magicDamageDealt_win': magicDamageDealt_win,
            'wardsKilled_win': wardsKilled_win,
            'largestCriticalStrike_win': largestCriticalStrike_win,
            'doubleKills_win': doubleKills_win,
            'tripleKills_win': tripleKills_win,
            'magicDamageDealtToChampions_win': magicDamageDealtToChampions_win,
            'totalTimeCrowdControlDealt_win': totalTimeCrowdControlDealt_win,
            'physicalDamageTaken_win': physicalDamageTaken_win,
            'largestKillingSpree_win': largestKillingSpree_win,
            'inhibitorKills_win': inhibitorKills_win,
            'physicalDamageDealtToChampions_win': physicalDamageDealtToChampions_win,
            'quadraKills_win': quadraKills_win,
            'goldSpent_win': goldSpent_win,
            'wardsPlaced_win': wardsPlaced_win,
            'killingSprees_win': killingSprees_win,
            'totalUnitsHealed_win': totalUnitsHealed_win,
            'totalPlayerScore_win': totalPlayerScore_win,
            'magicDamageTaken_win': magicDamageTaken_win,
            'largestMultiKill_win': largestMultiKill_win,
            'combatPlayerScore_win': combatPlayerScore_win,
            'combatPlayerScore_win': combatPlayerScore_win
            }
        return res
        ## << TO DO >>
        # how many times he/she plays with the other sids in the selected match before this date ...

    # --------------------------------------------------------------------------------------
    
    def get_matches_champions_and_summonerIds_before_game(self, season):
        with open('summonerId_list.pkl', 'rb') as f:
            all_summoners = pk.load(f)
        summoners = [ all_summoners[i] for i in sorted(random.sample(range(len(all_summoners)), 3000)) ]
        for sid in summoners:
            matchids = self._get_list_of_match_ids(sid, season)
            for matchid in matchids:
                match = self._get_match(matchid)
                if 'participants' in match:
                    losers = []
                    winners = []
                    winners_sid = []
                    losers_sid = []
                    for member in match['participants']:
                        suId = [pi['player']['summonerId'] for pi in match['participantIdentities'] if pi['participantId']==member['participantId']][0]       
                        selected_champion_id = member['championId']
                        if member['stats']['winner']:
                            winners_sid.append(suId)
                            winners.append(self.get_all_history_in_same_season_before_this_date(suId, selected_champion_id, season, match['matchCreation']))
                        else:
                            losers_sid.append(suId)
                            losers.append(self.get_all_history_in_same_season_before_this_date(suId, selected_champion_id, season, match['matchCreation']))
                    stats = {}
                    return winners, losers
                # << TO DO >> CHECK IF WINNER OR LOSER IS {}
                
#                     for k in winners[0].keys():
#                         a = []
#                         for w in winners:
#                             a.append(w[k])
#                         stats['winner_'+k+'_mean'] = np.average(a)
#                         stats['winner_'+k+'_std'] = np.std(a)
#                         a = []
#                         for l in losers:
#                             a.append(l[k])
#                         stats['loser_'+k+'_mean'] = np.average(a)
#                         stats['loser_'+k+'_std'] = np.std(a)
#                     data = {'matchId': match['matchId'], 'duration': match['matchDuration'], 'stats': stats, 'summoners': winners_sid + losers_sid}
#                     yield data
    
        
                    
with open('../MyKey4.key', 'r') as key_file:
    KEY = key_file.readline().strip()
api = RiotAPI(KEY)

In [82]:
#g = api.get_all_history_in_same_season_before_this_date(5, 1, 2016, 1453951950351)

In [ ]:
SID = 46422404
for matchid in api._get_list_of_match_ids(SID, 2016, 1467349723067):
    match = api._get_match(matchid)
    if 'participants' in match:
        for pi in match['participantIdentities']:
            if pi['player']['summonerId'] == SID:
                for pa in match['participants']:
                    if pa['participantId'] == pi['participantId']:
                        if pa['teamId'] == 100:
                            print('WINNER')
                        else:
                            print('LOSER')

In [83]:
WINER,LOSER = api.get_matches_champions_and_summonerIds_before_game(season=2016)

KeyboardInterrupt: 

In [53]:
WINER[2]

{}

In [41]:
stats = {}
for k in list(WINER[0].keys()):
    a = []
    for w in WINER:
        print(w[k])
        a.append(w[k])
    stats['winner_'+k+'_mean'] = np.average(a)
    stats['winner_'+k+'_std'] = np.std(a)
    a = []
    for l in LOSER:
        a.append(l[k])
    stats['loser_'+k+'_mean'] = np.average(a)
    stats['loser_'+k+'_std'] = np.std(a)

1
1


KeyError: 'inhibitorKills_lose'

In [26]:
stats

{}

In [9]:
LIMIT = 1100
dt = api.get_matches_champions_and_summonerIds_before_game(season=2016)

data = []
for d in dt:
    data.append(d)
    if not len(data) % 5:
        print(len(data))
    if len(data) > LIMIT:
        break

KeyError: 'inhibitorKills_lose'

In [15]:
len(data)

0

In [ ]:
#dataset = pd.DataFrame(data=data, columns=list(data[0]['stats'].keys()))
#dataset.to_csv('FULL_PREVIOUS_Stat2016_for_Classification2016_Dataset.csv')